In [69]:
import numpy as np
import pandas as pd
from scipy.stats import norm
import yfinance as yf

### Problem 1.
We consider an N-period binomial market, with the notation used in the lectures. We
consider a special option of American type: it gives you the right to sell a share of stock for
the strike price $K$ at any time until expiration (similarly as an American put). However,
if you did not exercise earlier than $T$, you must exercise at time $T$ (and potentially get a
negative payoff). In short, if you exercise when the stock price is $S_t$, your payoff is $K −S_t$
(even if it’s negative). We call $X_0$ the price of this option at time 0.
#### (a) For the option above, prove using backward induction that it’s always better to exercise than to wait for another period. Thus, the optimal strategy is to exercise immediately and the price of this option is $X_0 = K − S_0$

Hint: use a similar idea as in the proof that optimal exercise for the American call is at maturity and the fact that, at any node $k$,
$q_k s_{2k+1} + (1 − q_k)s_{2k} = (1 + r\Delta t) s_k$.

#### (b) Let $C_0$ (resp. P0) denote the price of an American call (resp. put) with strike price K. We consider two investment opportunities:
1. an American put with strike price $K$;
2. an American call with strike price $K$ and the option studied above.

Explain why the second opportunity is at least as valuable as the first one. Conclude
that $P_0 \leq K − S_0 + C_0$.
#### (c) Using (European) put-call parity established in HW#2, conclude that $P_ 0 \geq \frac{K}{(1 + r \Delta t)^N}− S_0 + C_0$.

#### (d) The results of (b) and (c) provide bounds for the American put price, which use quantities which are easier to calculate (remember that the price of the American and European call are the same). Calculate those bounds for the market model in Problem 1 of HW#3 and comment on their quality.

#### (e) What happens if the interest rate is 0% ? Find the price of the options in the market model of Problem 1 of HW#3 if $r = 0%$. What can you conclude about the price of an American put when $r = 0%$?




### Problem 2. Fundamental option pricing formula
This problem is based on the fundamental option pricing formula for the CRR model
developed in class, namely the value at time 0 of an option with maturity T and payoff
F is given by: $F_0 = e^{−rT}E_Q[F]$.

We consider the two options below:

- A. An option with which you must buy a share of stock at expiration $T = 1$ for strike
price $K = S_0$.
- B. An option with which you must buy a share of stock at expiration $T = 1$ for strike
price K given by $K = \frac{1}{N} \sum_{j=0}^N S_{t_{j}}$.

(Note that both options can have negative payoffs.) We use a CRR model with N periods
to price these options. Assume that the interest rate on the money market is r.

#### (a) Using the fundamental option pricing formula, find the price of option A. (Hint: use the martingale properties developed in the lectures for the stock price process.)

#### (b) Using the fundamental option pricing formula, find the price of option B.

#### (c) Find the limit as $N \to \infty $ of the results in (a) and (b).

#### (d) Assuming the interest rate is very small $(r \simeq 0)$, use Taylor expansions to find the first order approximation in r of the two results in (c).
#### (e) Can we intuitively explain the result ?



### Problem 3. Black-Scholes comparison

We put ourselves in the setting of Prob. 3 of HW#3. There, we worked out (using a
computer) the price of a European call and a European put options for large N. Use
the Black-Scholes formula to find the price of the European call and put-call parity (or
the Black-Scholes formula for a put) to find the price of the European put of Prob. 3 of
HW#3. Compare the results with the prices obtained in HW#3 and comment on the
quality of the approximation.

In [21]:
"""Assumptions we will carry through this problem using the info above"""
n=60
u = .005
d = .003
s_0 = 1000
r = .05
k= 1025
T = 1
r = .05

In [66]:
def bsm_call(s,k,r,T,n,sigma_real=1):
    dt = T/n
    mu = (np.log(1+u) + np.log(1-d)) / (2*dt)
    sigma = (np.log(1+u) - mu*dt) / (np.sqrt(dt))
    d1 = (np.log(s/k) + (r + (sigma**2)/2)*T) / (sigma*np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)
    price = s * norm.cdf(d1) - k * np.exp(-r*T) * norm.cdf(d2)
    return price

In [50]:
def put_call_parity(s_0,r,T,k,option_price,type='p'):
    if type.lower() == 'p':
        # C -P = S_0 - k(e^(-rt))
        put_price=option_price + k*(np.exp(-r*T)) - s_0
        return put_price
        
put_call_parity(s_0,r,T,k,bsm_call(s=s_0,k = 1025,r = .05,T = 1,n=60),type='p')

3.5611175402698336

In [67]:
bsm_call_p = bsm_call(s=s_0,k = 1025,r = .05,T = 1,n=60)
put_price = put_call_parity(s_0,r,T,k,bsm_call(s=s_0,k = 1025,r = .05,T = 1,n=60),type='p')
print(f'BSM price of the call is ${round(bsm_call_p,3)}')
print(f'BSM price of the put is ${round(put_price,3)}')

BSM price of the call is $28.551
BSM price of the put is $3.561



### Problem 4. Estimating parameters and option pricing
The purpose of this problem is to price options based on real data with the Black-Scholes
model.
Consider your selected stock for the semester (used in Prob. 2 of HW#4) and the
same options considered in that problem. Remember that the parameters µ and σ were
already estimated in part (a) of that problem. (You are welcome to consider the dates
and parameters obtained then, or reestimate the data with more up-to-date data, it’s up
to you.)

#### (a) Price the call options using the Black-Scholes formula, using the parameters estimated from your data.

In [72]:
"""Stock data""" 
visa = yf.Ticker('V')
v_historical = yf.download('V',start = '2021-01-01', end = '2022-03-03' ).drop(['Open','High','Low','Close'],axis = 1)
current_price = v_historical['Adj Close'][-1]

"""10 year yield"""
r_10y = yf.download('^tnx',start='2022-03-01')
r_10y = np.round(r_10y['Adj Close'][0],3)
r_dec_10y = r_10y/100
print(f'We use the rate (3/3/2022): {r_10y} %')

"""Estimate Params"""
delta_t = 1/252
v_historical['log_returns'] = np.log(1+(v_historical['Adj Close'].pct_change()))

sigma_v_sq = np.var(v_historical.log_returns.dropna()) / delta_t
sigma_v = np.sqrt(sigma_v_sq)
mu_v = np.mean(v_historical.log_returns.dropna()) /delta_t

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
We use the rate (3/3/2022): 1.707 %


#### (b) Compare your results to the market price and comment on the accuracy of your results and what could explain the differences.

#### (c) Repeat question (a) and (b) for the put options, either using the Black-Scholes formula for a put or the put-call parity formula. Comment on the accuracy of your results and explain any discrepancies between your estimate and the market price.